<a href="https://colab.research.google.com/github/MNagaSaiGanesh/Human-Stress-Detection/blob/main/Predictions_and_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from sklearn.preprocessing import StandardScaler
import joblib

# Assuming X_train is the data you used for training
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Save the scaler for later use
joblib.dump(scaler, 'scaler.pkl')

# Now save your trained model
model.save('stress_detection_model.h5')


In [14]:
import joblib
import tensorflow as tf
import numpy as np
from flask import Flask, request, jsonify
import warnings

# Suppress specific warning if you're only making predictions
warnings.filterwarnings("ignore", category=UserWarning, message="Compiled the loaded model.*")

# Load the trained model and scaler
model = tf.keras.models.load_model('stress_detection_model.h5')

# Recompile the model (only if you need metrics, otherwise you can skip this)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load the scaler (make sure this file exists and was saved earlier)
scaler = joblib.load('scaler.pkl')

# Initialize Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get input data from the request
    input_data = np.array(data['features']).reshape(1, -1)  # Reshape to 2D array

    # Apply the same scaling transformation to the input data
    input_data_scaled = scaler.transform(input_data)

    # Make a prediction
    prediction = model.predict(input_data_scaled)

    # Convert to binary prediction (0 or 1)
    stress_level = (prediction > 0.5).astype(int)

    return jsonify({'stress_level': stress_level[0][0]})

if __name__ == '__main__':
    # Start the Flask application
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
